In [ ]:
%matplotlib inline

This notebook is based on:

https://mne.tools/stable/auto_tutorials/stats-sensor-space/75_cluster_ftest_spatiotemporal.html

# Spatiotemporal permutation F-test on full sensor data

Tests for differential evoked responses in at least
one condition using a permutation clustering test.
The FieldTrip neighbor templates will be used to determine
the adjacency between sensors. This serves as a spatial prior
to the clustering. Spatiotemporal clusters will then
be visualized using custom matplotlib code.

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

from mne.stats import spatio_temporal_cluster_test
from mne.channels import find_ch_adjacency
from mne.viz import plot_compare_evokeds

## Read epochs



Your pipeline from the previous notebook(s) should go here. Basically from reading the raw to the epoching needs to be done here.

Once it is all epoched you can continue. _Remember to equalise your conditions!_

The MNE-python stats functions work on a numpy array with the shape: 

- n_observations $\times$ n_times $\times$ n_channels/n_vertices

So we need to extract the data and then transform it to the right shape. _Remember_ MNE-python epochs are in the shape:

- n_observations $\times$ n_channels/n_verticies $\times$ n_times

n_channels/n_verticies is because the functions works both on sensor space and source space data.


You should also select just two conditions, e.g. left vs right auditory or auditory vs visual.

In [ ]:
X = [epochs[k].get_data() for k in event_dict]  # as 3D matrix
X = [np.transpose(x, (0, 2, 1)) for x in X]  # transpose for clustering

## Find the FieldTrip neighbor definition to setup sensor adjacency



In [ ]:
adjacency, ch_names = find_ch_adjacency(epochs.info, ch_type='eeg')

print(type(adjacency))  # it's a sparse matrix!

plt.imshow(adjacency.toarray(), cmap='gray', origin='lower',
           interpolation='nearest')
plt.xlabel('{} EEG'.format(len(ch_names)))
plt.ylabel('{} EEG'.format(len(ch_names)))
plt.title('Between-sensor adjacency')

## Compute permutation statistic

How does it work? We use clustering to "bind" together features which are
similar. Our features are the magnetic fields measured over our sensor
array at different times. This reduces the multiple comparison problem.
To compute the actual test-statistic, we first sum all F-values in all
clusters. We end up with one statistic for each cluster.
Then we generate a distribution from the data by shuffling our conditions
between our samples and recomputing our clusters and the test statistics.
We test for the significance of a given cluster by computing the probability
of observing a cluster of that size. For more background read:
Maris/Oostenveld (2007), "Nonparametric statistical testing of EEG- and
MEG-data" Journal of Neuroscience Methods, Vol. 164, No. 1., pp. 177-190.
doi:10.1016/j.jneumeth.2007.03.024



## TASK

Look up what the different parameters in the function does!

In [ ]:
# set family-wise p-value
p_accept = 0.05

cluster_stats = spatio_temporal_cluster_test(X, n_permutations=2000,
                                             threshold=None, tail=0,
                                             n_jobs=1, buffer_size=None,
                                             adjacency=adjacency)

T_obs, clusters, p_values, _ = cluster_stats
good_cluster_inds = np.where(p_values < p_accept)[0]

Note. The same functions work with source estimate. The only differences
are the origin of the data, the size, and the adjacency definition.
It can be used for single trials or for groups of subjects.

## Visualize clusters



**Adjust the visualization to the conditions you have selected!**

In [ ]:
# configure variables for visualization
colors = {"Aud": "crimson", "Vis": 'steelblue'}
linestyles = {"L": '-', "R": '--'}

# organize data for plotting
evokeds = {cond: epochs[cond].average() for cond in event_id}

# loop over clusters
for i_clu, clu_idx in enumerate(good_cluster_inds):
    # unpack cluster information, get unique indices
    time_inds, space_inds = np.squeeze(clusters[clu_idx])
    ch_inds = np.unique(space_inds)
    time_inds = np.unique(time_inds)

    # get topography for F stat
    f_map = T_obs[time_inds, ...].mean(axis=0)

    # get signals at the sensors contributing to the cluster
    sig_times = epochs.times[time_inds]

    # create spatial mask
    mask = np.zeros((f_map.shape[0], 1), dtype=bool)
    mask[ch_inds, :] = True

    # initialize figure
    fig, ax_topo = plt.subplots(1, 1, figsize=(10, 3))

    # plot average test statistic and mark significant sensors
    f_evoked = mne.EvokedArray(f_map[:, np.newaxis], epochs.info, tmin=0)
    f_evoked.plot_topomap(times=0, mask=mask, axes=ax_topo, cmap='Reds',
                          vmin=np.min, vmax=np.max, show=False,
                          colorbar=False, mask_params=dict(markersize=10))
    image = ax_topo.images[0]

    # create additional axes (for ERF and colorbar)
    divider = make_axes_locatable(ax_topo)

    # add axes for colorbar
    ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
    plt.colorbar(image, cax=ax_colorbar)
    ax_topo.set_xlabel(
        'Averaged F-map ({:0.3f} - {:0.3f} s)'.format(*sig_times[[0, -1]]))

    # add new axis for time courses and plot time courses
    ax_signals = divider.append_axes('right', size='300%', pad=1.2)
    title = 'Cluster #{0}, {1} sensor'.format(i_clu + 1, len(ch_inds))
    if len(ch_inds) > 1:
        title += "s (mean)"
    plot_compare_evokeds(evokeds, title=title, picks=ch_inds, axes=ax_signals,
                         colors=colors, linestyles=linestyles, show=False,
                         split_legend=True, truncate_yaxis='auto')

    # plot temporal cluster extent
    ymin, ymax = ax_signals.get_ylim()
    ax_signals.fill_betweenx((ymin, ymax), sig_times[0], sig_times[-1],
                             color='orange', alpha=0.3)

    # clean up viz
    mne.viz.tight_layout(fig=fig)
    fig.subplots_adjust(bottom=.05)
    plt.show()

## Exercises

- What is the smallest p-value you can obtain, given the finite number of
  permutations?

